In [1]:
import cv2
import sounddevice as sd
import time

In [2]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
from concurrent.futures import ThreadPoolExecutor

import pandas as pd
import numpy as np

from hume import HumeStreamClient, StreamSocket
from hume.models.config import FaceConfig

In [3]:
API_KEY = "lfPNP8UacszfWWGONuZcJByhwVo8mHSpQlN4AvLzJe8OiTNe"
FILE_PATH = "videos/frustrated_trim.mp4"

In [4]:
def sort_emotions(emotions):
    # Sort emotions by scores
    emotions.sort(key=lambda x: x['score'],reverse=True)
    emotions

def sort_results(results):
    results.apply(lambda x: sort_emotions(x["emotions"]),axis=1)
    print(results)

In [5]:
def run_in_executor_blocking(func, *args):
    loop = asyncio.get_event_loop()
    return loop.run_in_executor(ThreadPoolExecutor(), func, *args)
    
async def humeCall():
    print("Hume Call Start")
    client = HumeStreamClient(API_KEY)
    config = FaceConfig(identify_faces=True)
    
    async with client.connect([config]) as socket:
        result =  await socket.send_file(FILE_PATH)
    
    print(f"File Path = {FILE_PATH}")
    

    # print(result)
    try:
        results = result["face"]["predictions"]
    except:
        print("No Faces Detected")
        results = {}

    # print(results)

    global extracted_results
    extracted_results = pd.json_normalize(results)
        
    sort_results(extracted_results)
    print(extracted_results)
    return extracted_results


In [6]:
def startWebcam():
    # Open the webcam (you may need to change the argument to the appropriate camera index)
    cap = cv2.VideoCapture(0)
    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

    videoWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    videoHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    return cap, fourcc, videoWidth, videoHeight

In [7]:
async def start_video(video_id,fourcc, videoWidth, videoHeight):
    return cv2.VideoWriter(f'output_{video_id}.mp4', fourcc, 60.0, (videoWidth, videoHeight))

In [8]:
async def process_video(cap, fourcc, videoWidth, videoHeight):
    start_time = time.time()
    video_id = 0
    out = cv2.VideoWriter(f'output_{video_id}.mp4', fourcc, 60.0, (videoWidth, videoHeight))

    while cap.isOpened():
        ret, frame = cap.read()
        out.write(frame)
        cv2.imshow('Webcam Feed', frame)

        if time.time() - start_time > 5:
            out.release()
            print(f"Run: {video_id}")

            global FILE_PATH
            FILE_PATH = f"./output_{video_id}.mp4"
            print(FILE_PATH)
            
            video_id += 1
            start_time = time.time()

            humeTask = await asyncio.gather(start_video(video_id, fourcc, videoWidth, videoHeight),humeCall())

        
            out = humeTask[0]

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()



In [9]:
cap, fourcc, videoWidth, videoHeight= startWebcam()
# Run the event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(await process_video(cap, fourcc, videoWidth, videoHeight))


Run: 0
./output_0.mp4
Hume Call Start
File Path = ./output_0.mp4
   frame  time      prob face_id  \
0     20  None  0.999999  face_0   
1     40  None  1.000000  face_0   
2     60  None  0.999999  face_0   

                                            emotions      bbox.x      bbox.y  \
0  [{'name': 'Joy', 'score': 0.8985961675643921},...  761.679932  446.007294   
1  [{'name': 'Joy', 'score': 0.9347872138023376},...  763.147400  438.703918   
2  [{'name': 'Joy', 'score': 0.9249942898750305},...  764.874634  448.086334   

       bbox.w      bbox.h  
0  320.373047  430.237091  
1  335.888123  454.477966  
2  330.084473  445.864410  
   frame  time      prob face_id  \
0     20  None  0.999999  face_0   
1     40  None  1.000000  face_0   
2     60  None  0.999999  face_0   

                                            emotions      bbox.x      bbox.y  \
0  [{'name': 'Joy', 'score': 0.8985961675643921},...  761.679932  446.007294   
1  [{'name': 'Joy', 'score': 0.9347872138023376},...

TypeError: An asyncio.Future, a coroutine or an awaitable is required